In [2]:
# Import Libraries
import numpy as np
import pandas as pd
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedShuffleSplit
from IPython.display import HTML, display
import tabulate

Using TensorFlow backend.


## Leer Dataset

Leemos el dataset correspondiente: 

- data_windows: Es el dataset con vectores de 528 características
- data_windows_264: Es el dataset con vectores de 264 características
- data_windows_132: Es el dataset con vectores de 132 características

In [16]:
#Dataset
dataset = pd.read_csv('dataset/data_windows_132.csv').iloc[:,1:]

In [17]:
dataset.shape

(29848, 133)

## Paso se preprocesamiento

Se eliminan vectores que contentan datos Nan.

In [35]:
#Delete NAN values
"""
dataset_x = []
dataset_y = []
for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        if (len(dataset_x) <= (26*label)):
            dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
            dataset_y.append(dataset.iloc[index,0:1].values)
        else:
            break
"""
dataset_x = []
dataset_y = []

count = np.zeros(17)

for label in range(1,17):
    data = dataset[dataset.iloc[:,0] == label]
    for index, row in data.iterrows():
        if (len(dataset_x) <= (810*label)):#Balancear las clases
            dataset_x.append(np.nan_to_num(dataset.iloc[index,1:].values))
            dataset_y.append(dataset.iloc[index,0:1].values)
            count[label]+=1
    
dataset_x = np.array(dataset_x)
dataset_y = np.array(dataset_y)

In [36]:
dataset_x.shape,dataset_y.shape

((12961, 132), (12961, 1))

In [37]:
print(count[1:])

[811. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810. 810.
 810. 810.]


In [38]:
#Normalice X values
mean = np.sum(dataset_x,axis=0,keepdims=True)/dataset_x.shape[0]
normal_variance = np.sum(dataset_x**2,axis=0,keepdims=True)/dataset_x.shape[0]
dataset_x_normalice = dataset_x-mean
dataset_x_normalice = dataset_x_normalice/np.sqrt(normal_variance+1e-8)

In [39]:
dataset_y_reshape = dataset_y.reshape(1,dataset_y.shape[0]).flatten().astype(int)
dataset_y_reshape[20:40]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [40]:
#Create Y binary class vector
y_train_vector = dataset_y_reshape
print(y_train_vector[0])
print(y_train_vector.shape)

1
(12961,)


## KFold
Creamos los diferentes grupos de indices K-Fold

In [43]:
#Dividimos en train y test
#kf = KFold(n_splits=4)
sss =  StratifiedShuffleSplit(n_splits=4,test_size=0.2, random_state=1)
#kf.get_n_splits(dataset_x_normalice)
sss.get_n_splits(dataset_x_normalice, y_train_vector)
index_kfol = []
for train_index, test_index in sss.split(dataset_x_normalice, y_train_vector):
    index_kfol.append([train_index,test_index])

## Metodo Test Configuration

El método admite un vector de valores para cada hiperparámetro a probar y ejecuta la prueba con 4 KFolds para cada configuración posible, finalmente imprime los resultados en una tabla.

In [44]:
def test_configuration(lr_v = [0.001], beta_1_v=[0.9],beta_2_v=[0.999] ,epochs_n = [10], batch_size_n = [32], input_shape_n = 132):
    
    configuration = 1
    table = [["Config","Train 1","Test 1","Train 2","Test 2","Train 3","Test 3","Train 4","Test 4","Train Prom","Test Prom"]]
    
    #Dividimos en train y test
    sss =  StratifiedShuffleSplit(n_splits=4,test_size=0.2,random_state=1)
    sss.get_n_splits(dataset_x_normalice, y_train_vector)
    index_kfol = []
    for train_index, test_index in sss.split(dataset_x_normalice, y_train_vector):
        index_kfol.append([train_index,test_index])
                        
    for batch in batch_size_n:
        for epoch in epochs_n:
            for beta2 in beta_2_v:
                for beta1 in beta_1_v:
                    for lr in lr_v:
                        
                        #For save accuracies
                        results = {}
                        
                        # Create Model
                        model = Sequential([
                            
                            Dense(128, activation='relu', input_shape=(input_shape_n,)),
                            Dense(64, activation='relu'),
                            Dense(32, activation='relu'),
                            Dense(16, activation='softmax'),
                        ])

                        #Compile Model
                        model.compile(
                          optimizer=Adam(learning_rate=lr, beta_1=beta1, beta_2=beta2, amsgrad=False),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'],
                        )
                        
                        Wsave = model.get_weights()

                        for g in range(4):

                            model.set_weights(Wsave)

                            #Get K-fold group 
                            train_index = index_kfol[g][0]
                            test_index = index_kfol[g][1]

                            x_train = dataset_x_normalice[train_index]
                            y_train = y_train_vector[train_index]
                            x_test = dataset_x_normalice[test_index]
                            y_test = y_train_vector[test_index]  

                            history = model.fit(
                              x_train, # training data
                              to_categorical(y_train-1), # training targets
                              epochs=epoch,
                              batch_size=batch,
                              verbose=0 #Desactivar imprimir cada epoca.
                            )

                            results[str(g)+"train"] = history.history['accuracy'][-1]

                            output = model.evaluate(x_test,to_categorical(y_test-1))

                            results[str(g)+"test"] = output[1]
                            
                        #Print Results
                        print("______________________Configuration-"+str(configuration)+"______________________")
                        print("LR:"+str(lr)+"- Beta_1:"+str(beta1)+"- Beta_2:"+str(beta2))
                        print("Epochs:"+str(epoch)+"- Bach:"+str(batch))
                        print("_________________________________________________________")
                        
                        prom_train = []
                        prom_test = []
                        for g in range(4):
                            prom_train.append(results[str(g)+"train"]*100)
                            prom_test.append(results[str(g)+"test"]*100)
                        
                        table.append([configuration,
                                      "%.2f" % round(results["0train"]*100,2),"%.2f" % round(results["0test"]*100,2),
                                      "%.2f" % round(results["1train"]*100,2),"%.2f" % round(results["1test"]*100,2),
                                      "%.2f" % round(results["2train"]*100,2),"%.2f" % round(results["2test"]*100,2),
                                      "%.2f" % round(results["3train"]*100,2),"%.2f" % round(results["3test"]*100,2),
                                      "%.2f" % round(sum(prom_train)/4,2),"%.2f" % round(sum(prom_test)/4,2)])
                        print("_______________________ENDTEST_________________________")   
                        configuration = configuration+1
                        
    display(HTML(tabulate.tabulate(table, tablefmt='html')))                                      
                        

In [45]:
lr_t=[0.001,0.00098,0.00096]
beta_1_t = [0.9]
beta_2_t = [0.999]
epochs_t = [12]
batch_size_t = [16,32]
test_configuration(lr_v=lr_t,beta_1_v=beta_1_t,beta_2_v=beta_2_t,epochs_n = epochs_t,batch_size_n = batch_size_t)

Epoch 1/12
10368/10368 [==============================] - 1s 49us/step - loss: 0.7317 - accuracy: 0.7953
Epoch 2/12
10368/10368 [==============================] - 0s 42us/step - loss: 0.2817 - accuracy: 0.9178
Epoch 3/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.1970 - accuracy: 0.9401
Epoch 4/12
10368/10368 [==============================] - 0s 42us/step - loss: 0.1627 - accuracy: 0.9495
Epoch 5/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.1341 - accuracy: 0.9566
Epoch 6/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.1116 - accuracy: 0.9653
Epoch 7/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.0987 - accuracy: 0.9675
Epoch 8/12
10368/10368 [==============================] - 0s 42us/step - loss: 0.0913 - accuracy: 0.9688
Epoch 9/12
10368/10368 [==============================] - 0s 42us/step - loss: 0.0758 - accuracy: 0.9738
Epoch 10/12
10368/10368 [==============================

10368/10368 [==============================] - 0s 43us/step - loss: 0.7780 - accuracy: 0.7837
Epoch 2/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.2708 - accuracy: 0.9222
Epoch 3/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.1913 - accuracy: 0.9406
Epoch 4/12
10368/10368 [==============================] - 0s 42us/step - loss: 0.1516 - accuracy: 0.9530
Epoch 5/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.1252 - accuracy: 0.9625
Epoch 6/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.1063 - accuracy: 0.9661
Epoch 7/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.0944 - accuracy: 0.9683
Epoch 8/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.0909 - accuracy: 0.9709
Epoch 9/12
10368/10368 [==============================] - 0s 43us/step - loss: 0.0756 - accuracy: 0.9740
Epoch 10/12
10368/10368 [==============================] - 0s 42us

Epoch 1/12
10368/10368 [==============================] - 0s 31us/step - loss: 0.8909 - accuracy: 0.7566
Epoch 2/12
10368/10368 [==============================] - 0s 26us/step - loss: 0.3141 - accuracy: 0.9080
Epoch 3/12
10368/10368 [==============================] - 0s 24us/step - loss: 0.2166 - accuracy: 0.9344
Epoch 4/12
10368/10368 [==============================] - 0s 24us/step - loss: 0.1673 - accuracy: 0.9506
Epoch 5/12
10368/10368 [==============================] - 0s 24us/step - loss: 0.1356 - accuracy: 0.9570
Epoch 6/12
10368/10368 [==============================] - 0s 24us/step - loss: 0.1155 - accuracy: 0.9636
Epoch 7/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.0924 - accuracy: 0.9694
Epoch 8/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.0862 - accuracy: 0.9710
Epoch 9/12
10368/10368 [==============================] - 0s 24us/step - loss: 0.0739 - accuracy: 0.9762
Epoch 10/12
10368/10368 [==============================

10368/10368 [==============================] - 0s 23us/step - loss: 0.8597 - accuracy: 0.7591
Epoch 2/12
10368/10368 [==============================] - 0s 24us/step - loss: 0.2927 - accuracy: 0.9119
Epoch 3/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.2029 - accuracy: 0.9385
Epoch 4/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.1579 - accuracy: 0.9519
Epoch 5/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.1273 - accuracy: 0.9592
Epoch 6/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.1071 - accuracy: 0.9664
Epoch 7/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.0916 - accuracy: 0.9704
Epoch 8/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.0852 - accuracy: 0.9717
Epoch 9/12
10368/10368 [==============================] - 0s 23us/step - loss: 0.0811 - accuracy: 0.9747
Epoch 10/12
10368/10368 [==============================] - 0s 23us

Config,Train 1,Test 1,Train 2,Test 2,Train 3,Test 3,Train 4,Test 4,Train Prom,Test Prom
1,98.05,94.95,98.08,95.06,98.16,96.07,97.95,94.79,98.06,95.22
2,97.89,94.87,97.87,95.80,98.02,95.03,98.27,94.87,98.01,95.14
3,97.95,94.72,97.96,95.80,98.27,94.95,98.30,95.03,98.12,95.12
4,98.10,94.87,98.23,95.10,97.74,96.14,98.19,95.41,98.06,95.38
5,97.60,95.03,97.96,95.91,98.15,95.03,98.13,95.14,97.96,95.28
6,97.86,95.10,98.16,94.37,98.04,95.22,98.09,95.18,98.04,94.97
